<a href="https://colab.research.google.com/github/sezinbiner/find_path_genetic_algorithms/blob/main/find_path.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
import numpy as np
import math
import random

moves=[[-1,0], [-1,-1], [0, -1], [1,-1], [1,0], [1, 1], [0, 1], [-1, 1]]
directions = [1,2,3,4,5,6,7,8];

def find_cost(old_move, new_move):
  if abs(old_move - new_move) == 1 or abs(old_move - new_move) == 7:
    return 1
  elif abs(old_move - new_move) == 2 or abs(old_move - new_move) == 6:
    return 2
  elif abs(old_move - new_move) == 3 or abs(old_move - new_move) == 5:
   return 3
  elif abs(old_move - new_move) == 0:
    return 0
  else:
    return 4


def create_first_route(start, matrix):
  #TODO yönü minimize etme
  end=[0,0]
  route = []
  cost = 0
  route.append(1)
  #end=start
  end[0]=start[0]
  end[1]=start[1]
  #print("start_fonicinde:",start)
  #print("end_fonicinde:",end)
  while len(route) <= 80:
    if end[0] == 0 and end[1] == 0:
      route.append(random.randint(5,7))
      print("end:",end)
      print(route[-1])
    elif end[0] == 8 and end[1] == 8:
      route.append(random.randint(1,3))
      print("end:",end)
      print(route[-1])
    elif end[0] == 8 and end[1] == 0:
      route.append(random.choice([1,7,8]))
      print("end:",end)
      print(route[-1])
    elif end[0] == 0 and end[1] == 8:
      route.append(random.randint(3,5))
      print("end:",end)
      print(route[-1])
    elif end[0] == 8:
      route.append(random.choice([3,2,1,7,8]))
      print("end:",end)
      print(route[-1])
    elif end[0] == 0:
      route.append(random.choice([3,4,5,6,7]))
    elif end[1] == 8:
      route.append(random.choice([1,2,3,4,5]))
    elif end[1] == 0:
      route.append(random.choice([5,6,7,8,1]))
    else:
      route.append(random.randint(1,8))
    
    cost += find_cost(route[-2], route[-1])
    add_move = moves[route[-1]-1]
    
    end[0] += add_move[0]
    end[1] += add_move[1]
    matrix[end[0], end[1]] = 1
   
  area = len(np.argwhere(matrix == 1))
 # print("end_fonksonunda",end)
  return cost, route, end, area

def try_routes(coming_routes, start, size):
  fitness_list = []
  return_routes = []
  end=[0,0]
  count_eklenmedi = 0
  count_eklendi = 0
  
  for route in coming_routes: 
    flag=0
    cost = 0            #population 5000
    matrix = np.zeros((size,size))
    matrix[start[0], start[1]] = 1
    end[0]=start[0]
    end[1]=start[1]
    for i in range (len(route)):       #length of one route 9*9-1
      if (i !=0):
        add_move =  moves[route[i]-1]
        end = [end[0] + add_move[0], end[1] + add_move[1]]
        if (end[0] <= 8) and (end[1] <= 8) and (end[0] >= 0) and (end[1] >= 0): #kaç kişi siliniyor bak az silinen varsa create route
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1
        else:  
          end = [end[0] - add_move[0], end[1] - add_move[1]]
          if end[0] == 0 and end[1] == 0:
            route[i]=random.randint(5,7)
          elif end[0] == 8 and end[1] == 8:
            route[i]=random.randint(1,3)
          elif end[0] == 8 and end[1] == 0:
            route[i]=random.choice([1,7,8])
          elif end[0] == 0 and end[1] == 8:
            route[i]=random.randint(3,5)
          elif end[0] == 8:
            route[i]=random.choice([3,2,1,7,8])
          elif end[0] == 0:
            route[i]=random.choice([3,4,5,6,7])
          elif end[1] == 8:
            route[i]=random.choice([1,2,3,4,5])
          elif end[1] == 0:
            route[i]=random.choice([5,6,7,8,1])
          add_move =moves[route[i]-1]
          end = [end[0] + add_move[0], end[1] + add_move[1]]
          cost += find_cost(route[i-1], route[i])
          matrix[end[0], end[1]] = 1  
    
    area = len(np.argwhere(matrix == 1))
    return_routes.append(route)
    fitness_list.append(find_fitness(start, end, cost, area))

  #print("return_routes",len(return_routes))
  return return_routes,fitness_list

def calculate_distance(start,end):
    dist = math.sqrt(((start[0]-end[0])**2)+((start[1]-end[1])**2))
    return dist

def find_fitness(start, end, cost, area):
  distance = calculate_distance(start, end)
  fitness = area - cost - distance
  return fitness

def cross_over(routes):
  new_generation = []

  count=0
  for i in range (len(routes)):

    route1 = random.choice(routes)
    route2 = random.choice(routes)
    #new_route1 = route1[0:21] + route2[21:41] + route1[41:61] + route2[61:]
    #new_route2 = route2[:21] + route1[21:41] + route2[41:61] + route1[61:]
    new_route1 = route1[:1]+route1[1:41] + route2[41:] 
    new_route2 = route2[:1]+route2[1:41] + route1[41:]      

    new_generation.append(new_route1)
    new_generation.append(new_route2)

  return new_generation

def mutation(routes, mutation_ratio, length, population):

  for i in range(int(mutation_ratio)):
    index = random.randint(0,population-1)
    route = routes[index]        #route sayısı çok azaldığı için hata alıyoruz
    for i in range (3):
      index2 = random.randint(1,length)
      direction = random.randint(1,8)
      route[index2] = direction
    routes[index] = route
  return routes

size = 9
population = 5000
selected = int(population/2)
generation = 200
length = size*size-1
mutation_ratio = 0.01
matrix = np.zeros((size,size))
start = [8,0]
end=[8,0]
#end = start
end[0]=start[0]
end[1]=start[1]
matrix[start[0], start[1]] = 1;
fitness_list = []
routes = []

for i in range (population):
  #print("start_once:",start)
  cost, route, end, area = create_first_route(start, matrix)
  #print("start:",start)
  #print("end",end)
  routes.append(route)
  fitness_list.append(find_fitness(start, end, cost, area))
  i += 1

zipped = list(zip(routes, fitness_list))
sorted_routes = sorted(zipped, key = lambda x: x[1])
unzipped_list = list(zip(*sorted_routes))
new_generation = unzipped_list[0]

#  populasyonun yarısını almalıyız almıyoruz?

#print("new gen:",len(new_generation))
#//TODO popülasyonu katlama

for i in range (40) : #(generation-1):
  cross_generation = cross_over(new_generation[0:selected])   
  #print("new gen:",len(cross_generation))
  mutated_generation = mutation(cross_generation, mutation_ratio*population, length, population)
  #print("muntanted gen:",len(mutated_generation))  
  new_routes, new_fitness = try_routes(mutated_generation, start, size)
  #print("new route_list lenght:", len(new_routes))
  zipped = list(zip(new_routes, new_fitness))
  sorted_routes = sorted(zipped, key = lambda x: x[1])
  unzipped_list = list(zip(*sorted_routes))
  new_generation = unzipped_list[0]

print("bitti")



Görüntülenen çıkış son 5000 satıra kısaltıldı.
7
end: [8, 3]
8
end: [8, 0]
7
end: [8, 1]
8
end: [8, 3]
3
end: [8, 2]
8
end: [8, 2]
2
end: [8, 1]
3
end: [8, 0]
7
end: [8, 1]
2
end: [8, 0]
7
end: [8, 1]
1
end: [8, 0]
7
end: [8, 1]
7
end: [8, 2]
3
end: [8, 1]
7
end: [8, 2]
3
end: [8, 1]
7
end: [8, 2]
3
end: [8, 1]
1
end: [8, 0]
7
end: [8, 1]
1
end: [8, 1]
7
end: [8, 2]
2
end: [8, 2]
7
end: [8, 3]
3
end: [8, 2]
7
end: [8, 3]
2
end: [8, 4]
8
end: [8, 7]
2
end: [8, 7]
7
end: [8, 8]
2
end: [8, 7]
7
end: [8, 8]
2
end: [8, 7]
7
end: [8, 8]
2
end: [8, 8]
1
end: [8, 8]
3
end: [8, 7]
7
end: [8, 0]
1
end: [8, 0]
1
end: [8, 0]
7
end: [8, 1]
2
end: [8, 3]
3
end: [8, 2]
8
end: [8, 4]
3
end: [8, 3]
2
end: [8, 3]
7
end: [8, 4]
8
end: [8, 6]
2
end: [8, 4]
1
end: [8, 3]
7
end: [8, 4]
2
end: [8, 0]
1
end: [8, 7]
1
end: [8, 8]
3
end: [8, 7]
7
end: [8, 8]
1
end: [8, 7]
1
end: [0, 0]
6
end: [8, 0]
8
end: [8, 0]
7
end: [8, 1]
8
end: [8, 3]
3
end: [8, 2]
3
end: [8, 1]
7
end: [8, 2]
2
end: [8, 4]
8
end: [8, 5]
1

In [108]:
print(str(new_generation[0]))
print(len(new_generation[0]))

[1, 1, 6, 2, 6, 2, 6, 2, 6, 3, 7, 2, 5, 8, 4, 7, 1, 8, 4, 6, 1, 3, 5, 2, 6, 3, 7, 7, 3, 3, 7, 3, 7, 3, 7, 3, 7, 3, 7, 1, 4, 8, 5, 3, 1, 5, 1, 6, 2, 7, 3, 7, 4, 7, 3, 7, 2, 5, 1, 7, 6, 2, 6, 3, 7, 3, 1, 5, 3, 8, 3, 6, 2, 6, 7, 3, 2, 7, 3, 8, 4]
81


In [109]:
end[0]=start[0]
end[1]=start[1]
print("start:",start)
print("end",end)
matrix = np.zeros((size,size))
for i in range (len(new_generation[0])-1):
  add_move =moves[new_generation[0][i+1]-1]
  end = [end[0] + add_move[0], end[1] + add_move[1]]
  print("end:",end)
print(end)
area = len(np.argwhere(matrix == 1))
print(area)


start: [8, 0]
end [8, 0]
end: [7, 0]
end: [8, 1]
end: [7, 0]
end: [8, 1]
end: [7, 0]
end: [8, 1]
end: [7, 0]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [7, 0]
end: [8, 0]
end: [7, 1]
end: [8, 0]
end: [8, 1]
end: [7, 1]
end: [6, 2]
end: [7, 1]
end: [8, 2]
end: [7, 2]
end: [7, 1]
end: [8, 1]
end: [7, 0]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [8, 2]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [7, 1]
end: [8, 0]
end: [7, 1]
end: [8, 1]
end: [8, 0]
end: [7, 0]
end: [8, 0]
end: [7, 0]
end: [8, 1]
end: [7, 0]
end: [7, 1]
end: [7, 0]
end: [7, 1]
end: [8, 0]
end: [8, 1]
end: [8, 0]
end: [8, 1]
end: [7, 0]
end: [8, 0]
end: [7, 0]
end: [7, 1]
end: [8, 2]
end: [7, 1]
end: [8, 2]
end: [8, 1]
end: [8, 2]
end: [8, 1]
end: [7, 1]
end: [8, 1]
end: [8, 0]
end: [7, 1]
end: [7, 0]
end: [8, 1]
end: [7, 0]
end: [8, 1]
end: [8, 2]
end: [8, 1]
end: [7, 0]
end: [7, 1]
end: [7, 0]
end: [6, 1]
end: [7, 0]
[7, 0]
0
